In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [18]:
# 1. 데이터 로드
data = pd.read_csv("D:/diabetes.csv")
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [19]:
# 2. 입력(X)과 타겟(y) 분리
X = data.drop(columns="Outcome")
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [20]:
y = data["Outcome"]  # 0 또는 1
y.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [21]:
# 3. 정규화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [22]:
# 4. 훈련/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((614, 8), (154, 8), (614,), (154,))

In [23]:
# 5. 텐서 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [24]:
# 6. 데이터 로더
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [25]:
# 7. 분류 모델 정의
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(8, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()  # 확률로 출력
        )

    def forward(self, x):
        return self.model(x)

In [26]:
# 8. 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ClassificationModel().to(device)
criterion = nn.BCELoss()  # Binary Cross Entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [27]:
# 9. 학습 루프
model.train()
for epoch in range(50):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

Epoch 1, Loss: 0.6519
Epoch 2, Loss: 0.6127
Epoch 3, Loss: 0.5569
Epoch 4, Loss: 0.5160
Epoch 5, Loss: 0.4773
Epoch 6, Loss: 0.4766
Epoch 7, Loss: 0.4622
Epoch 8, Loss: 0.4401
Epoch 9, Loss: 0.4463
Epoch 10, Loss: 0.4377
Epoch 11, Loss: 0.4285
Epoch 12, Loss: 0.4317
Epoch 13, Loss: 0.4228
Epoch 14, Loss: 0.4455
Epoch 15, Loss: 0.4176
Epoch 16, Loss: 0.4215
Epoch 17, Loss: 0.4304
Epoch 18, Loss: 0.4391
Epoch 19, Loss: 0.4167
Epoch 20, Loss: 0.4256
Epoch 21, Loss: 0.4334
Epoch 22, Loss: 0.4166
Epoch 23, Loss: 0.4048
Epoch 24, Loss: 0.4044
Epoch 25, Loss: 0.4075
Epoch 26, Loss: 0.3999
Epoch 27, Loss: 0.4075
Epoch 28, Loss: 0.4012
Epoch 29, Loss: 0.3993
Epoch 30, Loss: 0.4135
Epoch 31, Loss: 0.3990
Epoch 32, Loss: 0.3919
Epoch 33, Loss: 0.3851
Epoch 34, Loss: 0.3950
Epoch 35, Loss: 0.3833
Epoch 36, Loss: 0.4093
Epoch 37, Loss: 0.4019
Epoch 38, Loss: 0.3838
Epoch 39, Loss: 0.3845
Epoch 40, Loss: 0.3869
Epoch 41, Loss: 0.3726
Epoch 42, Loss: 0.3793
Epoch 43, Loss: 0.3833
Epoch 44, Loss: 0.37

In [28]:
# 10. 평가
model.eval()
all_preds, all_targets = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        preds = (outputs.cpu().numpy() > 0.5).astype(int)  # 0 또는 1
        all_preds.extend(preds)
        all_targets.extend(y_batch.numpy())

# 정확도 평가
accuracy = accuracy_score(all_targets, all_preds)
print(f"\nTest Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 76.62%
